In [2]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.init
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import gc

from tqdm.autonotebook import tqdm
import os
tqdm.pandas()

/usr/local/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
!nvidia-smi

Wed Jun 19 15:36:45 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  On   | 00000000:08:00.0 Off |                  N/A |
| 20%   48C    P2    51W / 250W |      1MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
train_dir = './train-data'
train_x = pd.read_csv(os.path.join(train_dir, 'train_x.csv'))
# Class will start with 0
train_y = pd.read_csv(os.path.join(train_dir, 'train_y.csv')) - 1
test_x = pd.read_csv(os.path.join(train_dir, 'test_x.csv'))

In [5]:
categorical_features = ['gender', 'city', 'prodName', 'color', 'carrier']

In [ ]:
train_x.fillna(train_x.mean(), inplace=True)
test_x.fillna(train_x.mean(), inplace=True)

In [ ]:
categories_dict = []
for cate in categorical_features:
    # (dict_size, embedding_size)
    dict_size = max(train_x[cate].unique()) + 1
    categories_dict.append((dict_size, min(dict_size * 2, 100)))

In [ ]:
categories_dict

[(2, 4), (364, 100), (227, 100), (136, 100), (4, 8)]

In [ ]:
class TabularDataset(Dataset):
    def __init__(self, x: pd.DataFrame, categories=None, y=None):

      self.num_samples = x.shape[0]

      if y is not None:
          # Train
          self.y = y.values.reshape(-1, 1)
      else:
          # Test
          self.y = np.zeros((self.num_samples, 1))

      self.cat_cols = categories
      self.cont_cols = [col for col in x.columns if col not in categories]

      self.x_cont = x[self.cont_cols].astype(np.float32).values

      self.x_cate = x[categories].astype(np.int64).values

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return [self.x_cont[idx], self.x_cate[idx], self.y[idx]]

![](images/nn-model.png)


In [ ]:
class TabularNN(nn.Module):
    def __init__(self, categories_dict, num_continuous, num_classes, layers, dropout):
        super().__init__()
        
        self.embeds = nn.ModuleList([nn.Embedding(x, y) for x, y in categories_dict])
        num_embed_out = sum([y for _, y in categories_dict])
        num_continuous = num_continuous
        self.num_classes = num_classes
        
        # FC
        fc1 = nn.Linear(num_embed_out + num_continuous, layers[0])
        self.fc_list = nn.ModuleList([fc1] + [nn.Linear(layers[i], layers[i+1]) 
                                              for i in range(len(layers) - 1)])
        
        for layer in self.fc_list:
            nn.init.kaiming_normal_(layer.weight.data, nonlinearity='relu')
            
        self.fc_out = nn.Linear(layers[-1], num_classes)
        nn.init.kaiming_normal_(self.fc_out.weight.data, nonlinearity='relu')
        
        # BN
        self.continuous_var_bn = nn.BatchNorm1d(num_continuous)
        self.bn_layers = nn.ModuleList([nn.BatchNorm1d(layer)
                                        for layer in layers])

        # Dropout
        self.embed_dropout = nn.Dropout(dropout)
        self.droput_layers = nn.ModuleList([nn.Dropout(dropout) for layer in layers])
        
    def forward(self, categorical, continuous):
        
        embeded = [layer(categorical[:, i]) for i, layer in enumerate(self.embeds)]
        embeded = torch.cat(embeded, 1)
        embeded = self.embed_dropout(embeded)
        
        norm_continuous = self.continuous_var_bn(continuous)
        
        # concat categorical and continuous data as input for fc
        x = torch.cat([embeded, norm_continuous], 1)
        
        for fc, bn, dropout in zip(self.fc_list, self.bn_layers, self.droput_layers):
            x = F.relu(fc(x))
            x = bn(x)
            x = dropout(x)
            
        return self.fc_out(x)
            

In [ ]:
net = TabularNN(categories_dict, train_x.shape[1] - len(categorical_features), 
                len(train_y['age_group'].unique()), [250, 100], 0.5)
net = net.to(device)
net

TabularNN(
  (embeds): ModuleList(
    (0): Embedding(2, 4)
    (1): Embedding(364, 100)
    (2): Embedding(227, 100)
    (3): Embedding(136, 100)
    (4): Embedding(4, 8)
  )
  (fc_list): ModuleList(
    (0): Linear(in_features=518, out_features=250, bias=True)
    (1): Linear(in_features=250, out_features=100, bias=True)
  )
  (fc_out): Linear(in_features=100, out_features=6, bias=True)
  (continuous_var_bn): BatchNorm1d(206, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn_layers): ModuleList(
    (0): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (embed_dropout): Dropout(p=0.5)
  (droput_layers): ModuleList(
    (0): Dropout(p=0.5)
    (1): Dropout(p=0.5)
  )
)

In [ ]:
def train(dataset, loader, net, optimizer):
    net.train()
    total_loss = 0
    for cont, cat, y in tqdm(loader, leave=False):
        cont = cont.to(device)
        cat = cat.to(device)
        y = y.long().flatten().to(device)
        optimizer.zero_grad()

        pred = net(cat, cont)
#         print(y.shape)
#         print(pred.shape)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss/float(len(dataset))

def predict(dataset, loader, net):
    net.eval()
    corrects = eval_loss = 0
    result = []
    for cont, cat, y in tqdm(loader):
        cont = cont.to(device)
        cat = cat.to(device)
        pred = net(cat, cont)
        
        result.append(torch.max(pred, 1)[1].view(y.size()).data)
    return result

def evaluate(dataset, loader, net):
    net.eval()
    corrects = eval_loss = 0

    for cont, cat, y in tqdm(loader):
        cont = cont.to(device)
        cat = cat.to(device)
        y = y.long().flatten().to(device)
        pred = net(cat, cont)
        loss = criterion(pred, y)
        
        eval_loss += loss.item()
        corrects += (torch.max(pred, 1)[1].view(y.size()).data == y.data).sum()
    #loss, correct count, accuracy
    return eval_loss/float(len(dataset)), corrects, corrects*100/len(dataset)

In [ ]:
import sklearn
import sklearn.model_selection
splits = 3
kfold = sklearn.model_selection.StratifiedKFold(splits, shuffle=True)

In [ ]:
batch_size = 502500
epoch = 24
lr = 0.1
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:
for train_index, test_index in kfold.split(train_x, train_y):
    net = TabularNN(categories_dict, train_x.shape[1] - len(categorical_features), 
                len(train_y['age_group'].unique()), [250, 120], 0.3)
    net = net.to(device)
    train_dataset = TabularDataset(train_x.iloc[train_index], categorical_features, train_y.iloc[train_index])
    train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
    validation_dataset = TabularDataset(train_x.iloc[test_index], categorical_features, train_y.iloc[test_index])
    validation_loader = DataLoader(validation_dataset, batch_size, shuffle=False)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    # learning rate decay
    scheduler=torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.3) 
    for i in tqdm(range(epoch)):
        print(train(train_dataset, train_loader, net, optimizer))
        print(evaluate(validation_dataset, validation_loader, net))
        scheduler.step()
    print("train acc:", evaluate(train_dataset, train_loader, net))
    del train_dataset
    del train_loader
    print("validation acc:", evaluate(validation_dataset, validation_loader, net))

4.32233216676696e-06


4.658907570211684e-06


4.051324946219962e-06


4.241084214181417e-06


3.961272847257897e-06


...

3.885265020117365e-06


3.7424096960089175e-06


3.8981270701262555e-06


3.1578951370943475e-06


2.991820351649742e-06
